In [1]:
import numpy as np
import matplotlib.pyplot as mp

In [2]:
N = []
CA = []
C = []

with open("1a6g.pdb", "r") as f:
    lines = f.readlines()
    for line in lines:
        k = line.split()
        if k[0] == 'ATOM':
            if k[3] == 'BASP' or k[3] =='BARG' or k[3] == 'BPHE':
                continue # we can understand this by manually looking at the "1a6g.pdb" file
            if k[2] == 'N':
                N.append([float(k[6]), float(k[7]), float(k[8])])
            elif k[2] == 'CA':
                CA.append([float(k[6]), float(k[7]), float(k[8])])
            elif k[2] == 'C':
                C.append([float(k[6]), float(k[7]), float(k[8])])

In [3]:
# for psi vals (N-CA-C-N)
psi_vals = []
psi_iters = len(N) - 1 # to prevent edge case
for i in range(psi_iters):
    vec1 = np.array([N[i][j] - CA[i][j] for j in range(3)])
    vec2 = np.array([CA[i][j] - C[i][j] for j in range(3)])
    vec3 = np.array([C[i][j] - N[i+1][j] for j in range(3)])
    crossproduct1 = np.cross(vec1, vec2)
    norm1 = crossproduct1/np.linalg.norm(crossproduct1)
    
    crossproduct2 = np.cross(vec2, vec3)
    norm2 = crossproduct2/np.linalg.norm(crossproduct2)
    
    dotproduct = np.dot(norm1, norm2)
    crossproduct = np.cross(norm1, norm2)
    sinofboth = np.dot(vec2, crossproduct)
    norm3 = sinofboth/np.linalg.norm(vec2)
    psi_vals.append((-1) * np.arctan2(norm3, dotproduct)*180/np.pi)
psi_vals.append(360.0)
# print(psi_vals)

In [4]:
# for phi_vals (C-N-CA-C)
phi_vals = []
phi_iters = len(C) - 1 # to prevent edge case
phi_vals.append(360.0)
for i in range(phi_iters):
    vec1 = np.array([C[i][j] - N[i+1][j] for j in range(3)])
    vec2 = np.array([N[i+1][j] - CA[i+1][j] for j in range(3)])
    vec3 = np.array([CA[i+1][j] - C[i+1][j] for j in range(3)])
    crossproduct1 = np.cross(vec1, vec2)
    norm1 = crossproduct1/np.linalg.norm(crossproduct1)
    crossproduct2 = np.cross(vec2, vec3)
    norm2 = crossproduct2/np.linalg.norm(crossproduct2)
    dotproduct = np.dot(norm1, norm2)
    crossproduct = np.cross(norm1, norm2)
    sinofboth = np.dot(vec2, crossproduct)
    norm3 = sinofboth/np.linalg.norm(vec2)
    phi_vals.append((-1) * np.arctan2(norm3, dotproduct)*180/np.pi)
# print(phi_vals)

## Q2 answer:

In [5]:
secondary_structure = {}
ss_rh = []
ss_rh_temp = []
ss_lh = []
ss_lh_temp = []
ss_ab = []
ss_ab_temp = []
ss_pb = []
ss_pb_temp = []
# For right handed alpha helices:
for i in range(151):
    if (-82 < phi_vals[i] < -32 and -72 < psi_vals[i] < -22):
        ss_rh_temp.append(i)
    elif len(ss_rh_temp) > 3:
        ss_rh.append(ss_rh_temp)
        ss_rh_temp = []
    else:
        ss_rh_temp = []
        
# For left handed alpha helices:   
for i in range(151):
    if (32 < phi_vals[i] < 82 and 22 < psi_vals[i] < 72):
        ss_lh_temp.append(i)
    elif len(ss_lh_temp) > 3:
        ss_lh.append(ss_lh_temp)
        ss_lh_temp = []
    else:
        ss_lh_temp = []

# For anti parallel beta sheets:
for i in range(151):
    if (-164 < phi_vals[i] < -114 and 110 < psi_vals[i] < 160):
        ss_ab_temp.append(i)
    elif len(ss_ab_temp) > 3:
        ss_ab.append(ss_ab_temp)
        ss_ab_temp = []
    else:
        ss_ab_temp = []

# For parallel beta sheets:
for i in range(151):
    if (-144 < phi_vals[i] < -94 and 88 < psi_vals[i] < 138):
        ss_pb_temp.append(i)
    elif len(ss_pb_temp) > 3:
        ss_pb.append(ss_pb_temp)
        ss_pb_temp = []
    else:
        ss_pb_temp = []
    
secondary_structure["R-alpha-helix"] = ss_rh
secondary_structure["L-alpha-helix"] = ss_lh
secondary_structure["Antiparallel-beta-sheet"] = ss_ab
secondary_structure["Parallel-beta-sheet"] = ss_pb
# print(secondary_structure["RH"][0][-1])
for i in range(len(secondary_structure["R-alpha-helix"])):
    # we add 1 to residue number as it is 1 based indexing
    print("Right handed alpha helix " + str(i+1) + ": from residue " + str(secondary_structure["R-alpha-helix"][i][0] + 1) + " to residue " + str(secondary_structure["R-alpha-helix"][i][-1] + 1))

Right handed alpha helix 1: from residue 4 to residue 17
Right handed alpha helix 2: from residue 21 to residue 35
Right handed alpha helix 3: from residue 37 to residue 40
Right handed alpha helix 4: from residue 52 to residue 56
Right handed alpha helix 5: from residue 59 to residue 76
Right handed alpha helix 6: from residue 83 to residue 94
Right handed alpha helix 7: from residue 101 to residue 117
Right handed alpha helix 8: from residue 125 to residue 148


## Q3 answer:

In [6]:
u = []
for i in range(len(secondary_structure["R-alpha-helix"])):
    u_is = []
    for j in range(len(secondary_structure["R-alpha-helix"][i])):
        if (j == 0 or j == len(secondary_structure["R-alpha-helix"][i]) - 1):
            continue
        else:
            r_iminus1 = CA[secondary_structure["R-alpha-helix"][i][j-1]]
            r_i = CA[secondary_structure["R-alpha-helix"][i][j]]
            r_iplus1 = CA[secondary_structure["R-alpha-helix"][i][j+1]]
            first_term = np.array(r_i) - np.array(r_iminus1)
            second_term = np.array(r_iplus1) - np.array(r_i)
            crossproduct = np.cross(first_term, second_term)
            norm = crossproduct/np.linalg.norm(crossproduct)
            u_is.append(norm)
    u.append(u_is)

In [7]:
H_2 = []
for i in range(len(secondary_structure["R-alpha-helix"])):
    N = len(secondary_structure["R-alpha-helix"][i]) - 3
    summation = 0
    for j in range(len(secondary_structure["R-alpha-helix"][i])):
        if (j == 0 or j == len(secondary_structure["R-alpha-helix"][i]) - 1 or j == len(secondary_structure["R-alpha-helix"][i]) - 2):
            continue
        else:
            dotproduct = np.dot(u[i][j-1], u[i][j])
            summation += dotproduct
    ans = (1/N) * summation
    H_2.append(ans)
    
print("H_2 values from helix 1 to 8 in a list:")
print(H_2)

H_2 values from helix 1 to 8 in a list:
[0.6226557494083846, 0.6666670255289425, 0.48249871808336825, 0.6487358479711358, 0.6392807322867116, 0.648204736980621, 0.6254530079714851, 0.6447985864327337]


In [8]:
H_3 = []
for i in range(len(secondary_structure["R-alpha-helix"])):
    N = len(secondary_structure["R-alpha-helix"][i]) - 2
    summation = 0
    for j in range(len(secondary_structure["R-alpha-helix"][i])):
        if (j == 0 or j == len(secondary_structure["R-alpha-helix"][i]) - 1):
            continue
        else:
            midindex = np.floor(len(secondary_structure["R-alpha-helix"][i]) / 2)
#             print(int(midindex))
            dotproduct = np.dot(u[i][j-1], u[i][int(midindex) - 1])
            summation += dotproduct
    ans = (1/N) * summation
    H_3.append(ans)
print("H_3 values from helix 1 to 8 in a list:")  
print(H_3)

H_3 values from helix 1 to 8 in a list:
[0.6625140629313011, 0.689657942484938, 0.7412493590416841, 0.7658238986474238, 0.6938094317613511, 0.7514449151561016, 0.6912696917631239, 0.7060058308930434]


In [9]:
H_4 = []
for i in range(len(secondary_structure["R-alpha-helix"])):
    N = len(secondary_structure["R-alpha-helix"][i]) - 2
    summation = 0
    for j in range(len(secondary_structure["R-alpha-helix"][i])):
        if (j == 0 or j == len(secondary_structure["R-alpha-helix"][i]) - 1):
            continue
        else:
            summation += u[i][j-1]
    ans = (1/N) * summation
    ans = np.dot(ans, ans)
    H_4.append(ans)
print("H_4 values from helix 1 to 8 in a list:")
print(H_4)

H_4 values from helix 1 to 8 in a list:
[0.6604200631220744, 0.7095793866628661, 0.7412493590416842, 0.7167259017921905, 0.687132093934439, 0.670115283952364, 0.6773680631056773, 0.6850390699343475]
